In [31]:
import os
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [32]:
os.chdir(r"C:\Users\SGU\Projects\mlops-with-mlflow")
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [33]:
import mlflow
from urllib.parse import urlparse
mlflow.get_tracking_uri()
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
tracking_url_type_store

'https'

In [34]:
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
os.getenv("MLFLOW_TRACKING_URI")

'https://dagshub.com/crazy-bot/mlops-with-mlflow.mlflow'

In [36]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [37]:
from mlProject.utils.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARAMS_FILE_PATH
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
        print(self.config)
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.target_columns
        
        create_directories([config.root_dir])
        return ModelEvaluationConfig(
            config.root_dir,
            config.test_data_path,
            config.model_path,
            config.metric_file,
            params,
            schema.name,
            mlflow_uri=os.getenv("MLFLOW_TRACKING_URI")
        )

In [38]:


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import pandas as pd
import os
from mlProject.utils.common import load_bin, save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config
        
    def eval_metrics(self, gt_y, pred_y):
        mae = mean_absolute_error(gt_y, pred_y)
        mse = mean_squared_error(gt_y, pred_y)
        r2 = r2_score(gt_y, pred_y)
        return mae, mse, r2
    
    def log_in_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        model = load_bin(self.config.model_path)
        
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            pred_y = model.predict(test_x)
            mae, mse, r2 = self.eval_metrics(test_y, pred_y)
            
            # save metrics in local metric file
            scores = {"mean_absolute_error": mae, "mean_squared_error": mse, "r2_score": r2}
            save_json(os.path.join(self.config.root_dir, self.config.metric_file), scores)
            
            # track in mlflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)
            
            if tracking_url_type_store != "file":
                # log and register in remote server
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNet")
            else:
                # log model in local
                mlflow.sklearn.log_model(model, "model")
        

In [39]:
try:
    cfg_mgr = ConfigurationManager()
    model_eval_config = cfg_mgr.get_model_evaluation_config()
    model_evaluator = ModelEvaluation(model_eval_config)
    model_evaluator.log_in_mlflow()
except Exception as e:
    raise e

[2023-08-12 12:33:27,231: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-12 12:33:27,235: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-12 12:33:27,239: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_path': 'artifacts/data_ingestion/winequality-red.csv', 'status_file': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'art

[2023-08-12 12:33:27,547: DEBUG: connectionpool: https://dagshub.com:443 "POST /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow/runs/create HTTP/1.1" 200 None]
[2023-08-12 12:33:27,563: INFO: common: json file: artifacts/model_evaluation\metric.json dumped successfully]
[2023-08-12 12:33:27,803: DEBUG: connectionpool: https://dagshub.com:443 "POST /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 None]
[2023-08-12 12:33:28,022: DEBUG: connectionpool: https://dagshub.com:443 "POST /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 None]


2023/08/12 12:33:33 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\SGU\AppData\Local\Temp\tmpf2gsn9f3\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.3.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


[2023-08-12 12:33:33,376: DEBUG: connectionpool: https://dagshub.com:443 "GET /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow/runs/get?run_uuid=9c8cf4935e7f4d38b3767230c6388e80&run_id=9c8cf4935e7f4d38b3767230c6388e80 HTTP/1.1" 200 None]
[2023-08-12 12:33:33,746: DEBUG: connectionpool: https://dagshub.com:443 "PUT /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow-artifacts/artifacts/fc26881db9aa4a1f9db986531e56014f/9c8cf4935e7f4d38b3767230c6388e80/artifacts/model/conda.yaml HTTP/1.1" 200 None]
[2023-08-12 12:33:34,071: DEBUG: connectionpool: https://dagshub.com:443 "PUT /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow-artifacts/artifacts/fc26881db9aa4a1f9db986531e56014f/9c8cf4935e7f4d38b3767230c6388e80/artifacts/model/MLmodel HTTP/1.1" 200 None]
[2023-08-12 12:33:34,479: DEBUG: connectionpool: https://dagshub.com:443 "PUT /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow-artifacts/artifacts/fc26881db9aa4a1f9db986531e56014f/9c8cf4935e7f4d38b3767230c6388e80/artifacts/model/model.pk

Successfully registered model 'ElasticNet'.


[2023-08-12 12:33:36,533: DEBUG: connectionpool: https://dagshub.com:443 "POST /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow/model-versions/create HTTP/1.1" 200 None]


2023/08/12 12:33:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNet, version 1
Created version '1' of model 'ElasticNet'.


[2023-08-12 12:33:36,749: DEBUG: connectionpool: https://dagshub.com:443 "POST /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow/runs/update HTTP/1.1" 200 None]
